In [ ]:
%%capture
!pip install keras==2.6.0
!pip install tensorflow==2.3.0
#!pip install tensorflow-gpu==2.3.0

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Pegando as bases

In [2]:
from subprocess import call
import pandas as pd
import tensorflow as tf
import sklearn.preprocessing as pp
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
print(tf.config.list_physical_devices())

In [ ]:
print(tf.__version__)

In [ ]:
gcs_folder_path = 'gs://analytics-labs/introduction_to_ann/datasets/'

In [ ]:
call(['gsutil', '-m', 'cp', gcs_folder_path + 'train.parquet','.'])
call(['gsutil', '-m', 'cp', gcs_folder_path + 'test.parquet','.'])

In [ ]:
datatypes = {'key': 'str', 
              'fare_amount': 'float32',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

In [3]:
train = pd.read_parquet('train.parquet').sample(frac=0.1)

In [ ]:
train = train.astype(dtype = datatypes)

In [ ]:
len(train)

In [4]:
## Retira null
train = train.dropna(how = 'any', axis = 'rows')
## Retira outliers
train= train[(train.dropoff_longitude < -72) & (train.dropoff_longitude > -75) &
             (train.dropoff_latitude < 42) & (train.dropoff_latitude > 40) &
             (train.pickup_longitude < -72) & (train.pickup_longitude > -75) &
             (train.pickup_latitude < 42) & (train.pickup_latitude > 40) &
             (train.passenger_count > 0) &
             (train.fare_amount < 250) & (train.fare_amount > 0)]
train = train[(train['dropoff_longitude'] != train['pickup_longitude'])]
train = train[(train['dropoff_latitude'] != train['pickup_latitude'])]

In [ ]:
len(train)

In [ ]:
539356*100/554239

In [ ]:
train.describe()

## Exploratória

In [ ]:
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot

In [ ]:
train = train.dropna(how = 'any', axis = 'rows')
train = train[train.fare_amount>0]
len(train)

In [ ]:
train2= train[(train.dropoff_longitude < -73.7) & (train.dropoff_longitude > -74.1) &
             (train.dropoff_latitude < 40.9) & (train.dropoff_latitude > 40.57) &
             (train.pickup_longitude < -73.7) & (train.pickup_longitude > -74.1) &
             (train.pickup_latitude < 40.9) & (train.pickup_latitude > 40.57) &
             (train.passenger_count > 0) &
             (train.fare_amount < 60) & (train.fare_amount > 0)]
len(train2)

In [ ]:
train2 = train.copy()

In [ ]:
for col in ['dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'passenger_count']:
    print(col)
    print(train2[col].max())
    print(train2[col].min())
    train2[col].hist(bins=100)
    plt.show()


In [ ]:
add_travel_vector_features(train2)
train2 = train2.dropna(how = 'any', axis = 'rows')
train2['pickup_datetime'] = pd.to_datetime(train2['pickup_datetime'])
add_time_variables(train2)

In [ ]:
for col in ['abs_diff_longitude', 'abs_diff_latitude', 'year', 'month', 'week', 'hour']:
    print(col)
    print(train2[col].max())
    print(train2[col].min())
    train2[col].hist(bins=100)
    plt.show()

In [ ]:
pyplot.scatter(train2.dropoff_longitude, train2.dropoff_latitude)
pyplot.show()

In [ ]:
pyplot.scatter(test.pickup_longitude, test.pickup_latitude)
pyplot.show()

In [ ]:
train2 = test.copy()
add_haversine_distance(train2)

print(len(train2))
#train2['ratio'] = train2['fare_amount']/train2['distance']
#train2= train2[(train2.distance < 40)]
#pyplot.scatter(train2.fare_amount, train2.distance)
train2['distance'].hist(bins=100)
pyplot.show()

In [ ]:
train2 = train.copy()
add_distance(train2)
train2= train2[(train2.distance > 0)]
train2['ratio'] = train2['fare_amount']/train2['distance']
train2= train2[(train2.ratio < 100)]
print(len(train2)*100/554239)
train2['ratio'].hist(bins=1000)
plt.show()

In [ ]:
train2.groupby('fare_amount').count().sort_values('key', ascending = False).head(100)

In [ ]:
train2 = train.copy()

In [ ]:
train2.groupby('fare_amount').count().head(20)

## Tratando base de treino

In [5]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [6]:
import math

def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
def add_time_variables(df):
    df['week'] = df['pickup_datetime'].dt.weekday
    df['hour'] = df['pickup_datetime'].dt.hour
    df['month'] = df['pickup_datetime'].dt.month
    df['year'] = df['pickup_datetime'].dt.year
    df['day'] = df['pickup_datetime'].dt.day
def late_night (df):
    df['late_night'] = 0
    df[(df['hour'] <= 6) | (df['hour'] >= 20)]['late_night'] = 1
def night (df):
    df['night'] = 0
    df[((df['hour'] <= 20) & (df['hour'] >= 16)) & (df['week'] < 5)]['night'] = 1

def add_haversine_distance(df):
    df['distance'] = haversine(df.pickup_latitude, df.pickup_longitude,
                               df.dropoff_latitude, df.dropoff_longitude)
def add_manhatan_distance(df):
    df['manhatan'] = df['abs_diff_longitude'] + df ['abs_diff_latitude']

def add_quadrant(df, n):
    """
    min_lat = min(train.pickup_latitude.min(), train.dropoff_latitude.min())
    min_lon = min(train.pickup_longitude.min(), train.dropoff_longitude.min())
    max_lat = max(train.pickup_latitude.max(), train.dropoff_latitude.max())
    max_lon = max(train.pickup_longitude.max(), train.dropoff_longitude.max())
    """
    min_lat = 40.6
    min_lon = -74.1
    max_lat = 40.9
    max_lon = -73.7
    cols = np.linspace(min_lat, max_lat, num=n)
    rows = np.linspace(min_lon, max_lon, num=n)
    df['pickup_quadrant_lat'] = np.searchsorted(cols, df['pickup_latitude'])
    df['pickup_quadrant_lon'] = np.searchsorted(rows, df['pickup_longitude'])
    df['pickup_quadrant'] = df['pickup_quadrant_lat'] + n*df['pickup_quadrant_lon']
    df['dropoff_quadrant_lat'] = np.searchsorted(cols, df['dropoff_latitude'])
    df['dropoff_quadrant_lon'] = np.searchsorted(rows, df['dropoff_longitude'])
    df['dropoff_quadrant'] = df['dropoff_quadrant_lat'] + n*df['dropoff_quadrant_lon']
    df['quadrant_distance'] = ((df['dropoff_quadrant_lat']-df['pickup_quadrant_lat'])^2+(df['dropoff_quadrant_lon']-df['pickup_quadrant_lon'])^2)
def add_direction(df):
    df['aux']=df.dropoff_longitude - df.pickup_longitude
    df['direction_longitude']=0
    df[df['aux']>0]['direction_longitude']=1
    df['aux']=df.dropoff_latitude - df.pickup_latitude
    df['direction_latitude']=0
    df[df['aux']>0]['direction_latitude']=1
    angle =(df.dropoff_latitude - df.pickup_latitude)/(df.dropoff_longitude - df.pickup_longitude)
    angle[(df.dropoff_longitude - df.pickup_longitude)==0]=0
    df['angle']=np.arctan(angle)
        

def add_special_location(df):
    locations = {'A' : [-74.0063889, 40.7141667],
                 'B' : [-73.7822222222, 40.6441666667],
                 'C' : [-74.175, 40.69],
                 'D' : [-73.87, 40.77]}
    for key in locations.keys():
        df['distance_dropoff_' + key] = (locations[key][1]-df['dropoff_latitude'])*(locations[key][1]-df['dropoff_latitude'])+(locations[key][0]-df['dropoff_longitude'])*(locations[key][0]-df['dropoff_longitude'])
        df['distance_pickup_' + key] = (locations[key][1]-df['pickup_latitude'])*(locations[key][1]-df['pickup_latitude'])+(locations[key][0]-df['pickup_longitude'])*(locations[key][0]-df['pickup_longitude'])
        

In [7]:
train_treated = train.copy()

In [8]:
train_treated = train_treated.dropna(how = 'any', axis = 'rows')
add_travel_vector_features(train_treated)
print("===")
train_treated['pickup_datetime'] = pd.to_datetime(train_treated['pickup_datetime'])
add_time_variables(train_treated)
print("===")
late_night(train_treated)
night(train_treated)
print("===")
add_haversine_distance(train_treated)
print("===")
add_manhatan_distance(train_treated)
print("===")
add_direction(train_treated)
print("===")
add_special_location(train_treated)
print("===")
add_quadrant(train_treated, 6)
print("===")
select_cols=['abs_diff_longitude', 'abs_diff_latitude', 'distance', 'passenger_count', 'fare_amount',
             'year', 'month', 'week', 'hour', 'day', 'pickup_quadrant', 'dropoff_quadrant',
             'distance_dropoff_A', 'quadrant_distance', 'pickup_quadrant_lat', 'pickup_quadrant_lon',
             'dropoff_quadrant_lat', 'dropoff_quadrant_lon',
             'distance_dropoff_B', 'distance_dropoff_C', 'distance_dropoff_D', 'distance_pickup_A',
             'distance_pickup_B', 'distance_pickup_C', 'distance_pickup_D', 'direction_longitude',
             'direction_latitude', 'angle', 'night', 'late_night', 'manhatan', 'dropoff_longitude',
             'pickup_longitude', 'dropoff_latitude', 'pickup_latitude']
encode_cols=['year', 'month', 'week', 'hour', 'pickup_quadrant', 'dropoff_quadrant']
encode_cols_dict={'week': 'week',
                  'hour': 'hour',
                  'month': 'month',
                  'year': 'year',
                  'pickup_quadrant': 'pickup_quadrant',
                  'dropoff_quadrant': 'dropoff_quadrant'}
train_treated = train_treated[select_cols]
train_treated = pd.get_dummies(train_treated, prefix=encode_cols_dict, columns=encode_cols)


===
===


<ipython-input-6-d52e71553a2d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(df['hour'] <= 6) | (df['hour'] >= 20)]['late_night'] = 1
<ipython-input-6-d52e71553a2d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[((df['hour'] <= 20) & (df['hour'] >= 16)) & (df['week'] < 5)]['night'] = 1


===
===
===


<ipython-input-6-d52e71553a2d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['aux']>0]['direction_longitude']=1
<ipython-input-6-d52e71553a2d>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['aux']>0]['direction_latitude']=1


===
===
===


In [9]:
y_train = train_treated[['fare_amount']]
X_train = train_treated.drop('fare_amount', axis=1)

In [10]:
scaler = pp.MinMaxScaler()
X_train = scaler.fit_transform(X_train)

scaler = pp.StandardScaler()
X_train = scaler.fit_transform(X_train)

## Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train,
    y_train,
    train_size=0.8,
    random_state=42)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    train_size=0.8,
    random_state=42)

## Treinando

In [12]:
### rede
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dropout(rate=0.06))
for _ in range(5):
    ann.add(tf.keras.layers.Dense(units=32, activation='elu'))
    ann.add(tf.keras.layers.Dropout(rate=0.06))
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))
adam = tf.keras.optimizers.Adam(learning_rate=0.0002)
ann.compile(optimizer = adam, loss = 'mse', metrics = tf.keras.metrics.RootMeanSquaredError())

In [13]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)

In [ ]:
history = ann.fit(X_train, y_train,
                  epochs = 200,
                  batch_size = 512,
                  validation_data=(X_val, y_val),
                  callbacks=early_stop)

Epoch 1/200
8355/8355 [==============================] - 34s 4ms/step - loss: 26.6424 - root_mean_squared_error: 5.1616 - val_loss: 14.7375 - val_root_mean_squared_error: 3.8389
Epoch 2/200
8355/8355 [==============================] - 33s 4ms/step - loss: 15.8459 - root_mean_squared_error: 3.9807 - val_loss: 13.5422 - val_root_mean_squared_error: 3.6800
Epoch 3/200
8355/8355 [==============================] - 34s 4ms/step - loss: 14.9255 - root_mean_squared_error: 3.8633 - val_loss: 13.2085 - val_root_mean_squared_error: 3.6344
Epoch 4/200
8355/8355 [==============================] - 35s 4ms/step - loss: 14.4849 - root_mean_squared_error: 3.8059 - val_loss: 12.9283 - val_root_mean_squared_error: 3.5956
Epoch 5/200
8355/8355 [==============================] - 35s 4ms/step - loss: 14.2444 - root_mean_squared_error: 3.7742 - val_loss: 12.7772 - val_root_mean_squared_error: 3.5745
Epoch 6/200
8355/8355 [==============================] - 33s 4ms/step - loss: 14.0522 - root_mean_squared_erro

Epoch 47/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.8722 - root_mean_squared_error: 3.5878 - val_loss: 12.3425 - val_root_mean_squared_error: 3.5132
Epoch 48/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.8613 - root_mean_squared_error: 3.5863 - val_loss: 12.2522 - val_root_mean_squared_error: 3.5003
Epoch 49/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.8349 - root_mean_squared_error: 3.5826 - val_loss: 12.2982 - val_root_mean_squared_error: 3.5069
Epoch 50/200
8355/8355 [==============================] - 34s 4ms/step - loss: 12.8411 - root_mean_squared_error: 3.5835 - val_loss: 12.2943 - val_root_mean_squared_error: 3.5063
Epoch 51/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.8049 - root_mean_squared_error: 3.5784 - val_loss: 12.2815 - val_root_mean_squared_error: 3.5045
Epoch 52/200
8355/8355 [==============================] - 33s 4ms/step - loss: 12.8245 - root_mean_square

Epoch 93/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.5639 - root_mean_squared_error: 3.5446 - val_loss: 12.1150 - val_root_mean_squared_error: 3.4807
Epoch 94/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.5418 - root_mean_squared_error: 3.5414 - val_loss: 12.1202 - val_root_mean_squared_error: 3.4814
Epoch 95/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.5257 - root_mean_squared_error: 3.5392 - val_loss: 12.1787 - val_root_mean_squared_error: 3.4898
Epoch 96/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.5512 - root_mean_squared_error: 3.5428 - val_loss: 12.1181 - val_root_mean_squared_error: 3.4811
Epoch 97/200
8355/8355 [==============================] - 32s 4ms/step - loss: 12.5281 - root_mean_squared_error: 3.5395 - val_loss: 12.1835 - val_root_mean_squared_error: 3.4905
Epoch 98/200
8355/8355 [==============================] - 31s 4ms/step - loss: 12.5423 - root_mean_square

#### Loop

In [ ]:
for x in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    print(x)
    ann = tf.keras.models.Sequential()
    ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
    ann.add(tf.keras.layers.Dropout(rate=0.06))
    for _ in range(5):
        ann.add(tf.keras.layers.Dense(units=32, activation='elu'))
        ann.add(tf.keras.layers.Dropout(rate=0.06))
    ann.add(tf.keras.layers.Dense(units=1, activation='linear'))
    adam = tf.keras.optimizers.Adam(learning_rate=0.0002)
    ann.compile(optimizer = adam, loss = 'mse', metrics = tf.keras.metrics.RootMeanSquaredError())
    
    ann.fit(X_train, y_train,
                  epochs = 50,
                  batch_size = 512,
                  validation_data=(X_val, y_val))
    y_pred = ann.predict(X_test)
    m = tf.keras.metrics.RootMeanSquaredError()
    m.update_state(y_test, y_pred)
    print(m.result().numpy())

In [ ]:
for x in [pp.MinMaxScaler(), pp.StandardScaler(), pp.PowerTransformer(method='yeo-johnson', standardize=True)]:
    print(x)
    """
    train2 = train_treated.copy()
    add_quadrant(train2, 6)
    select_cols=['abs_diff_longitude', 'abs_diff_latitude', 'distance', 'passenger_count', 'fare_amount',
                 'year', 'month', 'week', 'hour', 'day', 'pickup_quadrant', 'dropoff_quadrant', 'distance_dropoff_A',
                 'distance_dropoff_B', 'distance_dropoff_C', 'distance_dropoff_D', 'distance_pickup_A',
                 'distance_pickup_B', 'distance_pickup_C', 'distance_pickup_D', 'direction_longitude',
                 'direction_latitude', 'angle', 'night', 'late_night', 'manhatan', 'dropoff_longitude',
                 'pickup_longitude', 'dropoff_latitude', 'pickup_latitude']
    encode_cols=['year', 'month', 'week', 'hour', 'pickup_quadrant', 'dropoff_quadrant']
    encode_cols_dict={'week': 'week',
                      'hour': 'hour',
                      'month': 'month',
                      'year': 'year',
                      'pickup_quadrant': 'pickup_quadrant',
                      'dropoff_quadrant': 'dropoff_quadrant'}
    train2 = train2[select_cols]
    train2 = pd.get_dummies(train2, prefix=encode_cols_dict, columns=encode_cols)
    """
    y_train = train2[['fare_amount']]
    X_train = train2.drop('fare_amount', axis=1)
    scaler = x
    X_train = scaler.fit_transform(X_train)
    X_train, X_test, y_train, y_test = train_test_split(
        X_train,
        y_train,
        train_size=0.8,
        random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        train_size=0.8,
        random_state=42)
    ann = tf.keras.models.Sequential()
    ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
    ann.add(tf.keras.layers.Dropout(rate=0.1))
    for _ in range(4):
        ann.add(tf.keras.layers.Dense(units=32, activation='elu'))
        ann.add(tf.keras.layers.Dropout(rate=0.1))
    ann.add(tf.keras.layers.Dense(units=1, activation='linear'))
    adam = tf.keras.optimizers.Adam(learning_rate=0.0002)
    ann.compile(optimizer = adam, loss = 'mse', metrics = tf.keras.metrics.RootMeanSquaredError())
    
    ann.fit(X_train, y_train,
                  epochs = 50,
                  batch_size = 512,
                  validation_data=(X_val, y_val))
    y_pred = ann.predict(X_test)
    m = tf.keras.metrics.RootMeanSquaredError()
    m.update_state(y_test, y_pred)
    print(m.result().numpy())
    

## Fazendo Previsões e calculando métricas

In [ ]:
y_pred = ann.predict(X_test)

In [ ]:
m = tf.keras.metrics.RootMeanSquaredError()
m.update_state(y_test, y_pred)
m.result().numpy()

3.19

## Analisando treino

In [ ]:
def plot_history(history):
    
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    plt.figure(figsize=(20,4))
    plt.xlabel('Epoch')
    plt.ylabel('Root Mean Squared Error')
    plt.plot(
        hist['epoch'],
        hist['root_mean_squared_error'],
        label='Train Error')
    plt.plot(
        hist['epoch'],
        hist['val_root_mean_squared_error'],
        label = 'Val Error')
    plt.xlim([150,200])
    plt.ylim([3.4,3.5])
    plt.grid(alpha=0.25)
    plt.legend()

    plt.figure(figsize=(20,4))
    plt.xlabel('Epoch')
    plt.ylabel('Loss (mse)')
    plt.plot(
        hist['epoch'],
        hist['loss'],
        label='Train Loss')
    plt.plot(
        hist['epoch'],
        hist['val_loss'],
        label = 'Val Loss')
    #plt.ylim([20,30])
    plt.legend()
    plt.grid(alpha=0.25)
    plt.show()

In [ ]:
plot_history(history)

## Base de teste

In [ ]:
test = pd.read_parquet('test.parquet')

In [ ]:
datatypes = {'key': 'str', 
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

In [ ]:
test = test.astype(dtype = datatypes)

In [ ]:
test['base'] = 'test'
test = pd.concat([test, train])

In [ ]:
test = test.set_index('key')

In [ ]:
add_travel_vector_features(test)
print("===")
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'])
add_time_variables(test)
print("===")
late_night(test)
night(test)
print("===")
add_haversine_distance(test)
print("===")
add_manhatan_distance(test)
print("===")
add_quadrant(test, 6)
print("===")
add_direction(test)
print("===")
add_special_location(test)

In [ ]:
select_cols=['abs_diff_longitude', 'abs_diff_latitude', 'distance', 'passenger_count',
             'year', 'month', 'week', 'hour', 'day', 'pickup_quadrant', 'dropoff_quadrant', 'distance_dropoff_A',
             'distance_dropoff_B', 'distance_dropoff_C', 'distance_dropoff_D', 'distance_pickup_A',
             'distance_pickup_B', 'distance_pickup_C', 'distance_pickup_D', 'direction_longitude',
             'direction_latitude', 'angle', 'night', 'late_night', 'manhatan', 'base', 'dropoff_longitude',
             'pickup_longitude', 'dropoff_latitude', 'pickup_latitude', 'quadrant_distance',
             'pickup_quadrant_lat', 'pickup_quadrant_lon',
             'dropoff_quadrant_lat', 'dropoff_quadrant_lon',]
encode_cols=['year', 'month', 'week', 'hour', 'pickup_quadrant', 'dropoff_quadrant']
encode_cols_dict={'week': 'week',
                  'hour': 'hour',
                  'month': 'month',
                  'year': 'year',
                  'pickup_quadrant': 'pickup_quadrant',
                  'dropoff_quadrant': 'dropoff_quadrant'}
test = test[select_cols]
test = pd.get_dummies(test, prefix=encode_cols_dict, columns=encode_cols)

In [ ]:
test = test[test['base']=='test'].drop('base', axis=1)
drop_cols = [c for c in test.columns if c not in train_treated.columns]
print(drop_cols)
for col in drop_cols:
    test = test.drop(col, axis=1)

In [ ]:
key = test.reset_index().key

In [ ]:
test = scaler.transform(test)

In [ ]:
y_pred = ann.predict(test)

In [ ]:
y_pred = y_pred[:,0]

In [ ]:
submission = pd.DataFrame(
    {'key': key, 'fare_amount': y_pred},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)

In [ ]:
y_pred